In [1]:
import numpy as np
import os
from astropy.time import Time
import pandas as pd

In [3]:
def NOT_qu(kappa,err_kappa):

    k_0 = kappa[0]
    k_22 = kappa[1]
    k_45 = kappa[2]
    k_67 = kappa[3]

    ek_0 = err_kappa[0]
    ek_22 = err_kappa[1]
    ek_45 = err_kappa[2]
    ek_67 = err_kappa[3]

    aQ = np.sqrt(k_0/k_45)
    aU = np.sqrt(k_22/k_67)

    q = (1-aQ)/(1+aQ) #Q/I
    u = (1-aU)/(1+aU) #U/I

    q_ran = (aQ/((aQ + 1)**2)  *  np.sqrt(ek_0 + ek_45))
    u_ran = (aU/((aU + 1)**2)  *  np.sqrt(ek_22 + ek_67))
    q_sys = 0
    u_sys = 0
    
    return (q,q_ran,q_sys,u, u_ran, u_sys)

def efficiency(q,q_ran,q_sys,u, u_ran, u_sys,eff=None, efferr=None): 
    ###==================== 
    ## Correct Efficiency
    ###==================== 
    if eff==None:
        eff = 1
    if efferr==None:    
        efferr = 0

    qq = q/eff
    uu = u/eff

    #random error of corrected q,u
    qq_ran = q_ran/eff
    uu_ran = u_ran/eff

    #the systematic errors
    qq_sys = np.abs(q)*efferr/eff
    uu_sys = np.abs(u)*efferr/eff  
    
    return (qq,qq_ran,qq_sys,uu,uu_ran,uu_sys)
    

    
def Inst_pol(qq,qq_ran,qq_sys,uu,uu_ran,uu_sys, 
             q_inst = None, u_inst=None,eq_inst = None, eu_inst = None):
    
    ###==================== 
    ## Correc Instrumental polarization
    ###====================     
    if q_inst == None:
        q_inst =0.00018
    if u_inst == None:    
        u_inst =-0.00047
    if eq_inst == None:     
        eq_inst = 0.00037
    if eu_inst  == None:    
        eu_inst =  0.00065
    
    
    qqq = qq - q_inst
    uuu = uu - u_inst

    #random error of corrected q,u
    qqq_ran = qq_ran
    uuu_ran = uu_ran    

    #the systematic errors    
    qqq_sys = np.sqrt( qq_sys**2 + eq_inst**2)
    uuu_sys = np.sqrt( uu_sys**2 + eu_inst**2)    

    return(qqq,qqq_ran,qqq_sys,uuu,uuu_ran,uuu_sys)

def PositionANG_offset(qqq,qqq_ran,qqq_sys,uuu,uuu_ran,uuu_sys,
                      the=None,the_err=None):
    ###==================== 
    ## Transform_CelestialCoord
    ###====================    
    if the==None:
        the =   -87.70 # -92.30 #deg #the = Obs-Catal
    if the_err==None:    
        the_err =  0.10#0.060 #0.3
        
    theta = the
    theta = np.deg2rad(theta)
    the_err = np.deg2rad(the_err)

    qqqq = qqq * np.cos(2*theta) + uuu*np.sin(2*theta)
    uuuu = -qqq * np.sin(2*theta) + uuu*np.cos(2*theta)

    qqqq_ran = np.sqrt( (qqq_ran*np.cos(2*theta))**2 + (uuu_ran*np.sin(2*theta))**2 ) 
    uuuu_ran = np.sqrt( (qqq_ran*np.sin(2*theta))**2 + (uuu_ran*np.cos(2*theta))**2 ) 

    qqqq_sys = np.sqrt( (qqq_sys*np.cos(2*theta))**2 + \
                        (uuu_sys*np.sin(2*theta))**2 + \
                        (np.pi/180*2*uuuu*the_err)**2 )
    uuuu_sys = np.sqrt( (qqq_sys*np.sin(2*theta))**2 + \
                        (uuu_sys*np.cos(2*theta))**2 + \
                        (np.pi/180*2*qqqq*the_err)**2 ) 
#     print('Position angle offset correction is done.')
    
    return(qqqq, qqqq_ran, qqqq_sys, uuuu, uuuu_ran, uuuu_sys)

def weight(x,err):
    x = np.array(x)
    err = np.array(err)
    w = 1/err**2
    sumW = np.sum(w)
    weight = w/sumW
    xav = np.sum(weight*x)
    Err = 1/np.sqrt(sumW)
    return(xav,Err)

def theta_range(theta):
    if theta < -45:
        theta = theta+180
        return theta_range(theta)
    elif theta > 180:
        theta = theta-180
        return theta_range(theta)
    else:
        return (theta)


In [4]:
def make_Pol_file(phot,eff=True,inst=True,off=True):

    df = pd.read_csv(phot)
    order = np.arange(0,len(df),4)    
    FILE = df['filename'].values

    Pol_Log = pd.DataFrame({})
    for z in order:
        SET = [FILE[z],FILE[z+1], FILE[z+2 ], FILE[z+3]]
        kappa, err_kappa = [],[]

        JD_av = np.mean(df['JD'].values[z:z+4])
        psANG_av = np.mean(df['PsANG [deg]'].values[z:z+4])
        file_name = SET[0].split('/')[-1].split('.')[0] +' ~ '+SET[3].split('/')[-1].split('_')[-1]
        OBJECT = str(df['Object'].values[0])
        UT_av = Time(JD_av, format='jd').isot
        Date_obs = UT_av.split('T')[0]
        UT_obs = UT_av.split('T')[-1][:5]
        aperture_radius = np.mean(df['Aper_radius [pix]'].values[z:z+4])
        ap = np.mean(df['Aper_scale'].values[z:z+4])
        FWHM_o =  np.mean(df['FWHM_ordi [pix]'].values[z:z+4])
        FWHM_e = np.mean(df['FWHM_extra [pix]'].values[z:z+4])
        AIRMASS = np.mean(df['AIRMASS'].values[z:z+4])
        EXP = np.mean(df['EXPTIME'].values[z:z+4])
        pA = np.mean(df['alpha [deg]'].values[z:z+4])
        psANG = np.mean(df['PsANG [deg]'].values[z:z+4])
        snr = np.mean(df['SNR'].values[z:z+4])
        delta = np.mean(df['delta'].values[z:z+4])
        r = np.mean(df['r'].values[z:z+4])
        Type = df['Type'].values[0]
        FILTER = df['Filter'].values[0]
        eff_flag = False
        inst_flag = False
        off_flag = False


        for i, ang in enumerate([0,22.5,45,67.5]):
            file = SET[i]
            df_ = df[df['filename']==file]
            HWP = df_['HWPANGLE'].values[0]
            FILTER = df_['Filter'].values[0]
            if HWP not in [ang, ang+90, ang+180, ang+270]:
                print('HWPANG order is 0, 22.5,45,67.5?')
            Flux_extra = df_['Flux_extra [e]'].values[0]
            Flux_ordi = df_['Flux_ordi [e]'].values[0]
            ERR_extra = df_['eFlux_extra [e]'].values[0]
            ERR_ordi = df_['eFlux_ordi [e]'].values[0]
            kappa.append(Flux_extra/Flux_ordi)
            err_kappa.append( (Flux_extra/Flux_ordi**2 * ERR_ordi)**2 + (1/Flux_ordi * ERR_extra)**2 )
        

        qu = NOT_qu(kappa, err_kappa)
        if eff==True:
            qu = efficiency(*qu) #efficiency #q,q_ran,q_sys,u,uu_ran,u_sys
            eff_flag = True
        if inst==True:
            qu = Inst_pol(*qu) #Instrumental pol #q,q_ran,q_sys,u,u_ran,u_sys
            inst_flag = True
        if off==True:
            qu = PositionANG_offset(*qu) #Position angle offset transfor
            off_flag = True
            
        q, q_ran, q_sys, u, u_ran, u_sys  =qu
        eq, eu = (q_ran**2 + q_sys**2)**0.5,(u_ran**2 + u_sys**2)**0.5
        P = np.sqrt(q**2 + u**2)
        P_ran = np.sqrt( (q*q_ran)**2 + (u*u_ran)**2 )/P
        P_sys = np.sqrt( (q*q_sys)**2 + (u*u_sys)**2 )/P
        P_err = np.sqrt(P_ran**2 + P_sys**2) 

        theta_pol = np.rad2deg(1/2* np.arctan2(u,q))
        theta_ran = np.rad2deg(1/2 * P_ran/P)
        theta_sys = np.rad2deg(1/2 * P_sys/P)
        theta_err = np.sqrt(theta_ran**2 + theta_sys**2)  

        if psANG == -999:
            pi = -999
            theta_r = -999
        elif psANG + 90 < 180:
            pi = psANG + 90
            theta_r = theta_pol - pi
            theta_r = theta_range(theta_r)
        else:
            pi = psANG - 90
            theta_r = theta_pol - pi
            theta_r = theta_range(theta_r)


        if P**2 - P_ran**2 < 0:
            print('Due to P < randome error, random error bias correction is NOT done.')
            Pcor = 0
            theta_ran = 51.96
            theta_sys = 51.96
            theta_err = 51.96
        else:
            print('Random error bias correction is done.')
            Pcor = np.sqrt(P**2 - P_ran**2)
            theta_ran = np.rad2deg(1/2 * P_ran/Pcor)
            theta_sys = np.rad2deg(1/2 * P_sys/Pcor)
            theta_err = np.sqrt(theta_ran**2 + theta_sys**2) 

        if theta_r == -999:
            Pr = -999
            ePr = -999
        else:    
            Pr = Pcor * np.cos(2*np.deg2rad(theta_r))
            ePr = np.sqrt((np.cos(2*np.deg2rad(theta_r))*P_err)**2 + \
                          (2*Pcor*np.sin(np.deg2rad(2*theta_r))*np.deg2rad(theta_err))**2)

        Pol_Log = pd.concat([Pol_Log,
                             pd.DataFrame({
                                 'filename':[file_name],
                                  'Object':[OBJECT],
                                  'Type':[Type],
                                  'JD':[JD_av],
                                  'Filter':[FILTER],
                                  'DATE':[Date_obs],
                                  'UT':[UT_obs],
                                  'alpha [deg]':[pA],
                                  'PsANG [deg]':[psANG],
                                  'Aper_radius [pix]':[aperture_radius],
                                  'Aper_scale':[ap],
                                  'FWHM_o':[FWHM_o],
                                  'FWHM_e':[FWHM_e],
                                  'q':[q],
                                  'u': [u], 
                                  'eq':[eq],
                                  'eu':[eu],
                                  'ran_q':[q_ran],
                                  'ran_u':[u_ran],
                                  'sys_q':[q_sys],
                                  'sys_u':[u_sys],
                                  'theta':[theta_pol],
                                  'theta_r':[theta_r],
                                  'eTheta':[theta_err],
                                  'P [%]':[P*100],
                                  'P_cor [%]':[Pcor*100],
                                  'eP [%]':[P_err*100],
                                  'ePr [%]':[ePr*100],
                                  'Pr [%]':[Pr*100],
                                  'P_ran [%]':[P_ran*100],
                                  'P_sys [%]':[P_sys*100],
                                  'SNR':[snr],
                                  'AIRMASS':[AIRMASS],
                                  'delta':[delta],
                                  'r':[r],
                                  'EXPTIME':[EXP],
                                  'Eff_cor':[eff_flag],
                                  'Inst_cor':[inst_flag],
                                  'Off_cor':[off_flag]})])

    q_av, ranq_av = weight(Pol_Log['q'],Pol_Log['ran_q'])
    u_av, ranu_av = weight(Pol_Log['u'],Pol_Log['ran_u'])
    sysq_av = np.mean(Pol_Log['sys_q'])
    sysu_av = np.mean(Pol_Log['sys_u'])
    errq_av = (ranq_av**2 + sysq_av**2)**0.5
    erru_av = (ranu_av**2 + sysu_av**2)**0.5
    P = np.sqrt(q_av**2+u_av**2)
    ran_P = np.sqrt((q_av*ranq_av)**2 + (u_av*ranu_av)**2)/P
    sys_P = np.sqrt((q_av*sysq_av)**2 + (u_av*sysu_av)**2)/P
    eP = np.sqrt(ran_P**2 + sys_P**2)

    if P**2 - ran_P**2 < 0:
        Pcor = 0
        ran_PolAng = 51.96
        sys_PolAng = 51.96
        PolAng_error = 51.96
        # Naghizadeh-Khouei & Clarke 1993
    else:
        Pcor = np.sqrt(P**2 - ran_P**2)
        ran_PolAng = 1/2 * 180/3.14 * ran_P/Pcor
        sys_PolAng = 1/2 * 180/3.14 * sys_P/Pcor
        PolAng_error = np.sqrt(ran_PolAng**2 + sys_PolAng**2)  

    theta = 1/2*np.rad2deg(np.arctan2(u_av,q_av))
    psang = np.mean(Pol_Log['PsANG [deg]'])

    if psang == -999:
        pi = -999
        theta_r = -999
    elif psang + 90 < 180:
        pi = psang + 90
        theta_r = theta_pol - pi
        theta_r = theta_range(theta_r)
    else:
        pi = psang - 90
        theta_r = theta_pol - pi
        theta_r = theta_range(theta_r)
    if theta_r == -999:
        Pr = -999
        ePr = -999
    else:        
        Pr = Pcor*np.cos(2*np.deg2rad(theta_r))
        ePr = np.sqrt((np.cos(2*np.deg2rad(theta_r))*eP)**2 + 
                      (2*Pcor*np.sin(np.deg2rad(2*theta_r))*np.deg2rad(PolAng_error))**2)

    JD = np.mean(Pol_Log['JD'])
    alpha = np.mean(Pol_Log['alpha [deg]'])
    psang = np.mean(Pol_Log['PsANG [deg]'])
    snr_av = np.mean(Pol_Log['SNR'])
    delta_av = np.mean(Pol_Log['delta'])
    r_av = np.mean(Pol_Log['r'])
    UT_range = Pol_Log['UT'].values[0][:5] + '--'+Pol_Log['UT'].values[len(Pol_Log['UT'])-1][:5]
    
    AIRMASS_av = '{0:.1f}--{1:.1f}'.format(min(Pol_Log['AIRMASS'].values),max(Pol_Log['AIRMASS'].values))
    EXP_av = list(set(Pol_Log['EXPTIME'].values))
    if len(EXP_av)==1:
        EXP_av = EXP_av[0]

    file_name = 'Weighted_average'
    Pol_Log = pd.concat([Pol_Log,
                         pd.DataFrame(
                             {'filename':[file_name],
                              'Object':[OBJECT],
                              'Type':[Type],
                              'JD':[JD],
                              'Filter':[FILTER],
                              'DATE':[Date_obs],
                              'UT':[UT_range],
                              'alpha [deg]':[alpha],
                              'PsANG [deg]':[psang],
                              'q':[q_av],
                              'u':[u_av], 
                              'eq':[errq_av],
                              'eu':[erru_av],
                              'ran_q':[ranq_av],
                              'ran_u':[ranu_av],
                              'sys_q':[sysq_av],
                              'sys_u':[sysu_av],
                              'theta':[theta],
                              'theta_r':[theta_r],
                              'eTheta':[PolAng_error],
                              'P [%]':[P*100],
                              'P_cor [%]':[Pcor*100],
                              'eP [%]':[eP*100],
                              'ePr [%]':[ePr*100],
                              'Pr [%]':[Pr*100],
                              'P_ran [%]':[ran_P*100],
                              'P_sys [%]':[sys_P*100],
                              'SNR':[snr_av],
                              'AIRMASS':[AIRMASS_av],
                              'delta':[delta_av],
                              'r':[r_av],
                              'EXPTIME':[EXP_av],
                              'Aper_scale': [np.mean(df['Aper_scale'].values)],
                              'Eff_cor':[eff_flag],
                              'Inst_cor':[inst_flag],
                              'Off_cor':[off_flag]})])



    Pol_name = ['filename','Object','Type','DATE','UT','JD', 'Filter','alpha [deg]',
                'P [%]','P_cor [%]', 'Pr [%]', 'eP [%]','ePr [%]','theta_r','eTheta',
                'q', 'ran_q',  'sys_q','eq',
                'u','ran_u', 'sys_u', 'eu','theta',
                'AIRMASS', 'Aper_radius [pix]', 'Aper_scale', 'FWHM_e', 'FWHM_o', 
                'P_ran [%]', 'P_sys [%]','PsANG [deg]', 'SNR','delta','r','EXPTIME','flag','Type',
               'Eff_cor','Inst_cor','Off_cor']
    Pol_Log = Pol_Log.reindex(columns=Pol_name)

    Pol_Log = Pol_Log.round({'alpha [deg]':2,  'P [%]':2,'P_cor [%]':2, 
                             'Pr [%]':2, 'eP [%]':2,'theta_r':2,'eTheta':2,'P [%]':2, 'ePr [%]':2,
                             'q':4, 'ran_q':4,  'sys_q':4,'eq':4,
                             'u':4, 'ran_u':4, 'sys_u':4, 'eu':4,
                             'theta':2,'AIRMASS':2, 'Aper_radius [pix]':1,
                             'FWHM_e':2, 'FWHM_o':2, 
                             'P_ran [%]':2, 'P_sys [%]':2,'PsANG [deg]':2, 'SNR':1,
                             'delta':2,'r':2})

    Date_obs = Date_obs.replace('-','_')
    if os.path.exists(os.path.join('result'))==False:
        os.mkdir(os.path.join('result'))    
    Pol_Log.to_csv(os.path.join('result','Pol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)),
                  index=False)
    print(os.path.join('result','Pol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)))

    
def theta_range(theta):
    if theta < -45:
        theta = theta+180
        return theta_range(theta)
    elif theta > 180:
        theta = theta-180
        return theta_range(theta)
    else:
        return (theta)    
    
def weight(x,err):
    x = np.array(x)
    err = np.array(err)
    
    w = 1/err**2
    sumW = np.sum(w)
    weight = w/sumW
    
    xav = np.sum(weight*x)
    Err = 1/np.sqrt(sumW)
    
    return(xav,Err)

In [5]:
#The directory path where the photometric data fil
phot = 'Phot_2023_02_03_98943ap3.60.csv' #The photometric file generated by the 2.FAPOL_photometry.ipynb
make_Pol_file(phot,eff=True,inst=True,off=True)

Random error bias correction is done.
Random error bias correction is done.
result/Pol_98943.V_Bes 530_80.2023_02_03.ap3.60.csv
